In [ ]:
#!pip install packaging
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from packaging import version
import time

assert(version.parse(tf.__version__) >= version.parse("2.0.0-aplha"))

In [ ]:
def generate_serie(size, step) :
    maxT = size*step
    t = np.arange(0, maxT, step)
#    return (np.sin(t+np.cos(t*0.31))).astype(np.float32)
    return (np.sin(t+np.sin(t*.31)) * np.sin(t / 3.3485)).astype(np.float32)
#    return (np.sin(t+np.cos(t)) + .34158*np.sin(t * 3.3485)).astype(np.float32)
#return (t * np.sin(t)/maxT + .3*np.sin(t * 3.3)).astype(np.float32)

def split(arr, *count) :
    total = sum(count)
    p0 = 0
    for i in count :
        p1 = p0 + i
        yield arr[int(len(arr)*p0/total):int(len(arr)*p1/total)]
        p0 = p1
    
    
np.random.seed(42)

N_STEPS = n_steps = 50
data = generate_serie(14001, .18)
data = [(data[i:i+n_steps], data[i+1:i+n_steps+1]) for i in range(len(data)-n_steps-1)] 
np.random.shuffle(data)

#plt.plot(data[1][0])
#plt.plot(data[2][0])
#plt.plot(data[3][0])
#plt.show()

spt_data = np.reshape([data[i][0] for i in range(len(data))], (-1, n_steps, 1))
spt_lbl = np.reshape([data[i][1] for i in range(len(data))], (-1, n_steps, 1))

train_size = 80
valid_size = 15
test_size = 5

[train, valid, test] = zip(split(spt_data, train_size, valid_size, test_size), 
                           split(spt_lbl, train_size, valid_size, test_size))

plt.plot(train[0][1])
plt.plot(train[0][2])
plt.plot(train[0][3])
plt.show()

train = tf.data.Dataset.from_tensor_slices((train[0], train[1]))
valid = tf.data.Dataset.from_tensor_slices((valid[0], valid[1]))
test = tf.data.Dataset.from_tensor_slices((test[0], test[1]))
#train = train.batch(64, True)
#valid = valid.batch(64, True)
#test = test.batch(64, True)


In [ ]:
class ResetCallback(tf.keras.callbacks.Callback):
    def on_batch_begin(self, batch, logs=None) :
        print("on_batch_begin")
        
    def on_batch_end(self, batch, logs=None) :
        print("on_batch_end")

    def on_epoch_begin(self, epoch, logs=None) :
        print("on_epoch_begin")

    def on_epoch_end(self, epoch, logs=None) :
        print("on_epoch_end")

    def on_predict_batch_begin(self, batch, logs=None) :
        print("on_predict_batch_begin")

    def on_predict_batch_end(self, batch, logs=None) :
        print("on_predict_batch_end")

    def on_predict_begin(self, logs=None) :
        print("on_predict_begin")

    def on_predict_end(self, logs=None) :
        print("on_predict_end")

    def on_test_batch_begin(self, batch, logs=None) :
        print("on_test_batch_begin")
        self.model.reset_states()

    def on_test_batch_end(self, batch, logs=None) :
        print("on_test_batch_end")

    def on_test_begin(self, logs=None) :
        print("on_test_begin")
        self.model.reset_states()

    def on_test_end(self, logs=None) :
        print("on_test_end")

    def on_train_batch_begin(self, batch, logs=None) :
        print("on_train_batch_begin")
#        self.model.reset_states()

    def on_train_batch_end(self, batch, logs=None) :
        print("on_train_batch_end")

    def on_train_begin(self, logs=None) :
        print("on_train_begin")

    def on_train_end(self, logs=None) :
        print("on_train_end")

    def set_model(self, model) :
        self.model = model
        print("set_model")

    def set_params(self, params) :
        print("set_params", params)



In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, hidden1, enc_units, batch_sz,stateful=True, return_sequences=True) : #, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.hidden1 = hidden1
    self.enc_units = enc_units
    self.batch_sz = batch_sz
    #self.input_shape=(50, 1)

    #self.stateful = stateful
    self.lstm_stateful = stateful
    #self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    #self.inputi = tf.keras.Input(shape=(1,))
    #self.input_shape = tf.TensorShape([None,1])
#    self.d1 = tf.keras.layers.Dense(self.enc_units,
#                                    input_shape=(50, 1),
#                                    activation="linear")
    self.lstm = tf.keras.layers.LSTM(self.hidden1,
                                    return_sequences=return_sequences,
                                    #return_state=True,
                                    stateful=self.lstm_stateful,
                                    #activation="selu",
                                    #activation="linear",
                                    #recurrent_activation="selu",  
#                                    input_shape=(50, 1),
                                    recurrent_initializer='glorot_uniform')
#    self.dense = tf.keras.layers.TimeDistributed(
#        tf.keras.layers.Dense(self.enc_units,
#                              activation="linear"))
    self.dense = tf.keras.layers.Dense(self.enc_units,
                                    activation="linear")

    #self.gru = tf.keras.layers.RNN(
    #    tf.keras.layers.GRUCell(self.enc_units,
    #                               recurrent_initializer='glorot_uniform'),
    #   return_sequences=True,
    #   return_state=True
    #)
    
#  def call(self, x, initial_state=None): #, hidden):
  def call(self, x) : #, initial_state=None): #, hidden):
    #print(x, initial_state)
    #x2 = self.inputi(x)
    #print(x2)
#    if (initial_state!=None) :
#        self.lstm.reset_states(initial_state)
    
    #self.lstm.reset_states()
    #output = self.lstm(x) #, initial_state = hidden)
#    x = self.d1(x)
    x = self.lstm(x) #, initial_state = hidden)
    output = self.dense(x)
    #output, self.state = self.lstm(x, self.state) #, initial_state = hidden)
    return output #, state

  def reset_states(self) :
    self.lstm.reset_states()

#  def reset_states(self, initial_state=None) :
#    #print(self.lstm.input_spec)
#    self.lstm.reset_states(initial_state)

#  def initialize_hidden_state(self):
#    return tf.zeros((self.batch_sz, self.enc_units))
 

In [ ]:
BATCH_SIZE = 64*8
EPOCHS = 4

#encoder = keras.models.Sequential([
#    keras.layers.LSTM(20, return_sequences=True),
#    keras.layers.LSTM(20, return_sequences=True),
#    keras.layers.TimeDistributed(keras.layers.Dense(1))
#])
#print(3, N_STEPS, BATCH_SIZE)
encoder = Encoder(10, N_STEPS, BATCH_SIZE, False)
encoder.build(tf.TensorShape([BATCH_SIZE, None, 1]))
encoder.compile(optimizer="Adam", loss="MSE", metrics=['accuracy'])
#encoder.input_shape = tf.TensorShape([BATCH_SIZE, None, 1])
#print(encoder.input_shape)
encoder.summary()
encoder(tf.keras.Input(shape=(50, 1)))
print("stateful", encoder.stateful, encoder.state_updates)
print("encoder", encoder.input_shape, encoder.output_shape)
print("lstm", encoder.lstm.input_shape, encoder.lstm.output_shape)
print("dense", encoder.dense.input_shape, encoder.dense.output_shape)

encoder = Encoder(10, N_STEPS, BATCH_SIZE, False, return_sequences=False)
encoder.build(tf.TensorShape([BATCH_SIZE, None, 1]))
encoder.compile(optimizer="Adam", loss="MSE", metrics=['accuracy'])
#encoder.input_shape = tf.TensorShape([BATCH_SIZE, None, 1])
#print(encoder.input_shape)
encoder.summary()
encoder(tf.keras.Input(shape=(50, 1)))
print("stateful", encoder.stateful, encoder.state_updates)
print("encoder", encoder.input_shape, encoder.output_shape)
print("lstm", encoder.lstm.input_shape, encoder.lstm.output_shape)
print("dense", encoder.dense.input_shape, encoder.dense.output_shape)

encoder = Encoder(10, N_STEPS, BATCH_SIZE, True)
#encoder = Encoder(1, N_STEPS, BATCH_SIZE)
encoder.build(tf.TensorShape([BATCH_SIZE, None, 1]))
encoder.summary()
encoder.compile(optimizer="Adam", loss="MSE", metrics=['accuracy'])
encoder(tf.keras.Input(shape=(50, 1)))
print("stateful", encoder.stateful, encoder.state_updates)
print("encoder", encoder.input_shape, encoder.output_shape)
print("lstm", encoder.lstm.input_shape, encoder.lstm.output_shape)
print("dense", encoder.dense.input_shape, encoder.dense.output_shape)

encoder = Encoder(10, N_STEPS, BATCH_SIZE, True, return_sequences=False)
#encoder = Encoder(1, N_STEPS, BATCH_SIZE)
encoder.build(tf.TensorShape([BATCH_SIZE, None, 1]))
encoder.summary()
encoder.compile(optimizer="Adam", loss="MSE", metrics=['accuracy'])
encoder(tf.keras.Input(shape=(50, 1)))
print("stateful", encoder.stateful, encoder.state_updates)
print("encoder", encoder.input_shape, encoder.output_shape)
print("lstm", encoder.lstm.input_shape, encoder.lstm.output_shape)
print("dense", encoder.dense.input_shape, encoder.dense.output_shape)

#encoder.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
#print(len(train[0]))
#print(len(train[0])//BATCH_SIZE)
#print(len(train[0])//BATCH_SIZE*BATCH_SIZE)
#print(len(valid[0]))
#print(len(valid[0])//BATCH_SIZE)
#print(len(valid[0])//BATCH_SIZE*BATCH_SIZE)
#tbs = len(train[0])//BATCH_SIZE*BATCH_SIZE
#vbs = len(valid[0])//BATCH_SIZE*BATCH_SIZE

#a = encoder(train[0][:8])
#print(a)
#history = encoder.fit(train, epochs=EPOCHS) #, batch_size=BATCH_SIZE, shuffle=False)
#history = encoder.fit(train[0][:tbs], train[1][:tbs], epochs=EPOCHS, batch_size=BATCH_SIZE)
#history = encoder.fit(train.batch(BATCH_SIZE, True), epochs=EPOCHS, validation_data=valid.batch(BATCH_SIZE, True), shuffle=False) # , batch_size=BATCH_SIZE
#history = encoder.fit(train.batch(BATCH_SIZE, True), epochs=EPOCHS, validation_data=valid.batch(BATCH_SIZE, True), callbacks=[ResetCallback()], shuffle=False) # , batch_size=BATCH_SIZE
#history = encoder.fit(train.batch(64, True), epochs=EPOCHS, validation_data=valid.batch(64, True), callbacks=[ResetCallback()], shuffle=False) # , batch_size=BATCH_SIZE
#history = encoder.fit(train[0][:tbs], train[1][:tbs], epochs=EPOCHS, validation_data=(valid[0][:vbs], valid[1][:vbs]), batch_size=BATCH_SIZE)
#a = encoder(train[0][:8])
#print(a)


In [ ]:
#print(history)
#print(history.history)
print(history.history.keys())

plt.plot(history.history['loss'], label="loss")
plt.plot(history.history['val_loss'], label="val_loss")
plt.legend()
plt.show()
plt.plot(history.history['accuracy'], label="accuracy")
plt.plot(history.history['val_accuracy'], label="val_accuracy")
plt.legend()
plt.show()

In [ ]:
encoder.evaluate(test.batch(BATCH_SIZE, True))